# Data Collection and Cleaning

In [1]:
#imports
import requests
import time
import pandas as pd
import json
from pandas.io.json import json_normalize
import time

apikey = 'OOFXXAG7cuVc2q9C0EekCNqWm4lzERfm'

**Getting data for January 2019 from "archive" endpoint:**

In [2]:
resp = requests.get('https://api.nytimes.com/svc/archive/v1/2019/1.json?api-key=%s' %apikey)
archive_json = resp.json()
archive_json; #shows the full json but is very large

In [3]:
archive = json_normalize(archive_json['response']['docs']) #this is the highest level of the hierarchy
archive.columns

Index(['_id', 'blog', 'byline.organization', 'byline.original',
       'byline.person', 'document_type', 'headline.content_kicker',
       'headline.kicker', 'headline.main', 'headline.name',
       'headline.print_headline', 'headline.seo', 'headline.sub', 'keywords',
       'lead_paragraph', 'multimedia', 'news_desk', 'print_page', 'pub_date',
       'score', 'section_name', 'slideshow_credits', 'snippet', 'source',
       'subsectoinName', 'type_of_material', 'uri', 'web_url', 'word_count'],
      dtype='object')

In [4]:
archive["pub_date_str"] = archive["pub_date"].str.slice(0,10)
archive["pub_date_str"].head()

0    2019-01-03
1    2019-01-02
2    2019-01-02
3    2019-01-02
4    2019-01-02
Name: pub_date_str, dtype: object

In [5]:
archive.to_csv("archive.csv")

**Getting state data with "article search" endpoint:**

In [6]:
my_key = apikey

In [7]:
co_df = pd.DataFrame()
import time
for i in range(1,11):
    resp = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=headline:("Colorado")&page=%d&api-key=%s' % (i,my_key))
    headlines=resp.json()
    co_df = pd.concat([co_df, pd.DataFrame(json_normalize(headlines["response"]["docs"])["headline.main"])],
                        ignore_index=True)
    time.sleep(1.0)
time.sleep(60)

In [8]:
wa_df = pd.DataFrame()
import time
for i in range(1,11):
    resp = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=headline:("Washington")&page=%d&api-key=%s' % (i,my_key))
    headlines=resp.json()
    wa_df = pd.concat([wa_df, pd.DataFrame(json_normalize(headlines["response"]["docs"])["headline.main"])],
                        ignore_index=True)
    time.sleep(1.0)
time.sleep(60)

In [11]:
ny_df = pd.DataFrame()
import time
for i in range(1,11):
    resp = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=headline:("New York")&page=%d&api-key=%s' % (i,my_key))
    headlines=resp.json()
    ny_df = pd.concat([ny_df, pd.DataFrame(json_normalize(headlines["response"]["docs"])["headline.main"])],
                        ignore_index=True)
    time.sleep(1.0)
time.sleep(60)

In [13]:
ms_df = pd.DataFrame()
import time
for i in range(1,11):
    resp = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=headline:("Mississippi")&page=%d&api-key=%s' % (i,my_key))
    headlines=resp.json()
    ms_df = pd.concat([ms_df, pd.DataFrame(json_normalize(headlines["response"]["docs"])["headline.main"])],
                        ignore_index=True)
    time.sleep(1.0)
#time.sleep(60)

In [15]:
tx_df = pd.DataFrame()
import time
for i in range(1,11):
    resp = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=headline:("Texas")&page=%d&api-key=%s' % (i,my_key))
    headlines=resp.json()
    tx_df = pd.concat([tx_df, pd.DataFrame(json_normalize(headlines["response"]["docs"])["headline.main"])],
                        ignore_index=True)
    time.sleep(1.0)

In [16]:
co_df["state"] = "Colorado"
ny_df["state"] = "New York"
wa_df["state"] = "Washington"
ms_df["state"] = "Mississippi"
tx_df["state"] = "Texas"

head_df = pd.concat([co_df, ny_df, wa_df, ms_df, tx_df])

In [17]:
head_df["headline_raw"] = head_df["headline.main"].str.lower().replace("[^A-Za-z\s]", "").str.replace(";", "").replace(",", "")

head_df["head_split"] = head_df["headline_raw"].str.split(",").str.join("")

In [18]:
head_df.to_csv("state_headlines_for_model.csv")

**Get election data**

Election data from https://www.dailykos.com/stories/2016/11/25/1601042/-Nerd-Alert-This-spreadsheet-contains-every-presidential-election-by-state-from-1828-to-2016? which linked to https://docs.google.com/spreadsheets/d/1D-edaVHTnZNhVU840EPUhz3Cgd7m39Urx7HM8Pq6Pus/edit#gid=1024689221 which I downloaded to my computer and called "Raw_Election.csv"

In [19]:
election_df = pd.read_csv("Raw_Election.csv")
election_df = election_df.iloc[2:52, 0:3]

In [20]:
election_df.columns = ["State", "Dem%", "Rep%"]

election_df["isDem"] = pd.to_numeric(election_df["Dem%"]) > 50
election_df.to_csv("election_bool.csv")